In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
# Import Dependencies 
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Preprocess

In [2]:
# Read in Health Ranking Data
df = pd.read_csv("Data/County_Health_All_Factors.csv")
print(df.shape)
df.head()

(3142, 10)


,Unnamed: 0,FIPS,State,County,Lifespan_Rank,Life_Quality_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank
0,1,1001,Alabama,Autauga,8,5,15,14,5,50
1,2,1003,Alabama,Baldwin,3,4,3,4,3,62
2,3,1005,Alabama,Barbour,14,48,53,30,61,32
3,4,1007,Alabama,Bibb,47,24,38,16,38,31
4,5,1009,Alabama,Blount,36,14,10,41,15,53


In [3]:
# Read in the Merged Data
df2 = pd.read_csv("Data/Merged_Demographic_Data.csv")
print(df2.shape)
df2.head()

(3041, 25)


,County,State,FIPS,Lyme Disease Incidence Reported,Population,%<18_Yrs_Old,%65_Yrs_Old_and_over,Income($),African American(Count),African American(%),...,Native_Hawaiian/Other_Pacific_Islander(%),Hispanic(Count),Hispanic(%),Non-Hispanic_White(Count),Non-Hispanic_White(%),Female(%),Rural(Count),Rural(%),Life_Expectancy,Deaths(Count)
0,Autauga,Alabama,1001,1.0,55504,23.94,15.12,$58343.00,10687,19.25,...,0.10,1586,2.86,41336,74.47,51.34,22921.0,42.00,76.33,815.0
1,Baldwin,Alabama,1003,0.0,212628,21.85,19.95,$56607.00,19037,8.95,...,0.07,9675,4.55,176582,83.05,51.45,77060.0,42.28,78.60,2827.0
2,Barbour,Alabama,1005,1.0,25270,20.76,18.82,$32490.00,12115,47.94,...,0.19,1063,4.21,11613,45.96,47.23,18613.0,67.79,75.78,451.0
3,Bibb,Alabama,1007,0.0,22668,20.61,16.02,$45795.00,4864,21.46,...,0.11,598,2.64,16842,74.30,46.45,15663.0,68.35,73.93,445.0
4,Blount,Alabama,1009,0.0,58013,23.35,17.84,$48253.00,847,1.46,...,0.12,5549,9.57,50439,86.94,50.69,51562.0,89.95,74.60,1050.0


In [4]:
df_merged = df.merge(df2, how='left', on='FIPS')
print(df_merged.shape)
df_merged.head(10)

(3150, 34)


,Unnamed: 0,FIPS,State_x,County_x,Lifespan_Rank,Life_Quality_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank,...,Native_Hawaiian/Other_Pacific_Islander(%),Hispanic(Count),Hispanic(%),Non-Hispanic_White(Count),Non-Hispanic_White(%),Female(%),Rural(Count),Rural(%),Life_Expectancy,Deaths(Count)
0,1,1001,Alabama,Autauga,8,5,15,14,5,50,...,0.10,1586.0,2.86,41336.0,74.47,51.34,22921.0,42.00,76.33,815.0
1,2,1003,Alabama,Baldwin,3,4,3,4,3,62,...,0.07,9675.0,4.55,176582.0,83.05,51.45,77060.0,42.28,78.60,2827.0
2,3,1005,Alabama,Barbour,14,48,53,30,61,32,...,0.19,1063.0,4.21,11613.0,45.96,47.23,18613.0,67.79,75.78,451.0
3,4,1007,Alabama,Bibb,47,24,38,16,38,31,...,0.11,598.0,2.64,16842.0,74.30,46.45,15663.0,68.35,73.93,445.0
4,5,1009,Alabama,Blount,36,14,10,41,15,53,...,0.12,5549.0,9.57,50439.0,86.94,50.69,51562.0,89.95,74.60,1050.0
5,6,1011,Alabama,Bullock,53,58,65,62,64,24,...,0.78,850.0,8.25,2196.0,21.30,45.53,5607.0,51.37,73.12,205.0
6,7,1013,Alabama,Butler,63,56,56,52,55,8,...,0.05,285.0,1.44,10229.0,51.60,53.43,14921.0,71.23,73.51,393.0
7,8,1015,Alabama,Calhoun,55,10,28,34,27,41,...,0.11,4302.0,3.75,82961.0,72.31,51.93,39955.0,33.70,73.10,2333.0
8,9,1017,Alabama,Chambers,42,53,57,25,37,46,...,0.04,822.0,2.44,18710.0,55.50,52.13,16816.0,49.15,74.12,691.0
9,10,1019,Alabama,Cherokee,43,8,12,54,19,43,...,0.03,425.0,1.64,23707.0,91.69,50.32,22282.0,85.74,74.35,575.0


In [5]:
df_merged.set_index('FIPS', inplace =True)

In [6]:
df_merged = df_merged.dropna()

In [7]:
# Check for null values
for col in df_merged:
    print(f'{col} has {df_merged[col].isnull().sum()} nulls')

Unnamed: 0 has 0 nulls
State_x has 0 nulls
County_x has 0 nulls
Lifespan_Rank has 0 nulls
Life_Quality_Rank has 0 nulls
Health_Behaviors_Rank has 0 nulls
Clinical_Care_Rank has 0 nulls
Social_Economic_Factors_Rank has 0 nulls
Physical_Environment_Rank has 0 nulls
County_y has 0 nulls
State_y has 0 nulls
Lyme Disease Incidence Reported has 0 nulls
Population has 0 nulls
%<18_Yrs_Old has 0 nulls
%65_Yrs_Old_and_over has 0 nulls
Income($) has 0 nulls
African American(Count) has 0 nulls
African American(%) has 0 nulls
American_Indian/Alaskan_Native(Count) has 0 nulls
American_Indian/Alaskan_Native(%) has 0 nulls
Asian(Count) has 0 nulls
Asian(%) has 0 nulls
Native_Hawaiian/Other_Pacific_Islander(Count) has 0 nulls
Native_Hawaiian/Other_Pacific_Islander(%) has 0 nulls
Hispanic(Count) has 0 nulls
Hispanic(%) has 0 nulls
Non-Hispanic_White(Count) has 0 nulls
Non-Hispanic_White(%) has 0 nulls
Female(%) has 0 nulls
Rural(Count) has 0 nulls
Rural(%) has 0 nulls
Life_Expectancy has 0 nulls
Deaths

In [8]:
df_merged = df_merged.drop(columns=['Unnamed: 0','County_y','State_y','State_x','County_x'])

In [9]:
df_merged['Income($)'] = df_merged['Income($)'].str.replace('$', '')
df_merged['Income($)'] = df_merged['Income($)'].astype('float') / 100

C:\Users\efren\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [10]:
df_merged = df_merged.astype({'Lifespan_Rank': 'int32', 'Life_Quality_Rank': 'int32','Health_Behaviors_Rank':'int32',
'Clinical_Care_Rank':'int32','Social_Economic_Factors_Rank':'int32','Physical_Environment_Rank':'int32'})

In [11]:
df_merged.dtypes

Lifespan_Rank                                      int32
Life_Quality_Rank                                  int32
Health_Behaviors_Rank                              int32
Clinical_Care_Rank                                 int32
Social_Economic_Factors_Rank                       int32
Physical_Environment_Rank                          int32
Lyme Disease Incidence Reported                  float64
Population                                       float64
%<18_Yrs_Old                                     float64
%65_Yrs_Old_and_over                             float64
Income($)                                        float64
African American(Count)                          float64
African American(%)                              float64
American_Indian/Alaskan_Native(Count)            float64
American_Indian/Alaskan_Native(%)                float64
Asian(Count)                                     float64
Asian(%)                                         float64
Native_Hawaiian/Other_Pacific_I

## Split the Data into Training and Testing

In [12]:
# Create our features
X = df_merged.copy()
X = X.drop(columns='Lyme Disease Incidence Reported', axis=1)
           
# Create our target
y = df_merged[['Lyme Disease Incidence Reported']]
y.value_counts()

Lyme Disease Incidence Reported
0.0                                1886
1.0                                 314
2.0                                 147
3.0                                  81
4.0                                  57
                                   ... 
133.0                                 1
130.0                                 1
123.0                                 1
121.0                                 1
470.0                                 1
Length: 169, dtype: int64

In [39]:
# Create the bins in which Data will be held
# Bins are 0, 59, 69, 79, 89, 100.   
bins = [-1, 0, 500]

# Create the names for the four bins
group_names = ["0","1"]

In [41]:
y["Lyme_Cases"] = pd.cut(y["Lyme Disease Incidence Reported"], bins, labels=group_names)
y = y["Lyme_Cases"]

C:\Users\efren\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape

(2265, 27)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [49]:
# Resample the training data with the RandomOversampler

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'1': 1426, '0': 1426})

In [44]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=1)
classifier.fit(X_resampled, y_resampled)
y_pred = classifier.predict(X_test)

C:\Users\efren\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [45]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.7526455026455027


In [47]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report 

matrix = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    matrix, index=["Actual No Cases Reported", "Actual Cases Reported"], columns=["Predicted No Cases Reported", "Predicted Cases Reported"])
cm_df

,Predicted No Cases Reported,Predicted Cases Reported
Actual No Cases Reported,377,83
Actual Cases Reported,104,192


In [48]:
from imblearn.metrics import classification_report_imbalanced 
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.78      0.82      0.65      0.80      0.73      0.54       460
          1       0.70      0.65      0.82      0.67      0.73      0.52       296

avg / total       0.75      0.75      0.72      0.75      0.73      0.53       756



### SMOTE Oversampling

In [50]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)

In [51]:
# Train the Logistic Regression model using the resampled data
classifier = LogisticRegression(random_state=1)
classifier.fit(X_resampled, y_resampled)
y_pred = classifier.predict(X_test)

C:\Users\efren\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [52]:
# Calculated the balanced accuracy score
print(accuracy_score(y_test, y_pred))

0.7513227513227513


In [53]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report 

matrix = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    matrix, index=["Actual No Cases Reported", "Actual Cases Reported"], columns=["Predicted No Cases Reported", "Predicted Cases Reported"])
cm_df

,Predicted No Cases Reported,Predicted Cases Reported
Actual No Cases Reported,378,82
Actual Cases Reported,106,190


In [54]:
from imblearn.metrics import classification_report_imbalanced 
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.78      0.82      0.64      0.80      0.73      0.54       460
          1       0.70      0.64      0.82      0.67      0.73      0.52       296

avg / total       0.75      0.75      0.71      0.75      0.73      0.53       756



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [55]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'0': 839, '1': 839})

In [56]:
# Train the Logistic Regression model using the resampled data
classifier = LogisticRegression(random_state=1)
classifier.fit(X_resampled, y_resampled)
y_pred = classifier.predict(X_test)

C:\Users\efren\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [57]:
# Calculated the balanced accuracy score
print(accuracy_score(y_test, y_pred))

0.7513227513227513


In [58]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report 

matrix = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    matrix, index=["Actual No Cases Reported", "Actual Cases Reported"], columns=["Predicted No Cases Reported", "Predicted Cases Reported"])
cm_df

,Predicted No Cases Reported,Predicted Cases Reported
Actual No Cases Reported,376,84
Actual Cases Reported,104,192


In [59]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.78      0.82      0.65      0.80      0.73      0.54       460
          1       0.70      0.65      0.82      0.67      0.73      0.52       296

avg / total       0.75      0.75      0.71      0.75      0.73      0.53       756



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [60]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'0': 826, '1': 1001})

In [61]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = classifier.predict(X_test)

C:\Users\efren\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [62]:
# Calculated the balanced accuracy score
print(accuracy_score(y_test, y_pred))

0.7513227513227513


In [64]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report 

matrix = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    matrix, index=["Actual No Cases Reported", "Actual Cases Reported"], columns=["Predicted No Cases Reported", "Predicted Cases Reported"])
cm_df

,Predicted No Cases Reported,Predicted Cases Reported
Actual No Cases Reported,376,84
Actual Cases Reported,104,192


In [63]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.78      0.82      0.65      0.80      0.73      0.54       460
          1       0.70      0.65      0.82      0.67      0.73      0.52       296

avg / total       0.75      0.75      0.71      0.75      0.73      0.53       756

